# Trying to use pocketsphinx to word align

> "Because timing accuracy in ASR is getting progressively worse, look backwards"

- branch: master
- comments: false
- categories: [pocketsphinx, hsi, alignment]

In [1]:
from pydub import AudioSegment
import pocketsphinx
import tempfile

In [2]:
MAPPING = """
ɑː AA
æ AE
ə AH
ɐ AH
ʌ AH
ɔː AO
aʊ AW
aɪ AY
b B
tʃ CH
d D
ð DH
ɛ EH
ɚ ER
ɜː ER
eɪ EY
f F
ɡ G
h HH
ɪ IH
i IY
iː IY
dʒ JH
k K
l L
m M
n N
ŋ NG
oʊ OW
ɔɪ OY
p P
ɹ R
s S
ʃ SH
t T
θ TH
ʊ UH
uː UW
v V
w W
j Y
z Z
ʒ ZH
ɾ D
"""

In [3]:
espeak_to_cmudict = {}
for line in MAPPING.split("\n"):
    if line == "":
        continue
    line = line.strip()
    parts = line.split(" ")

    if len(parts) != 2:
        print(line)
        continue
    k, v = line.split(" ")
    if not k in espeak_to_cmudict:
        espeak_to_cmudict[k] = v
    

In [4]:
import re

cmudict_keys = espeak_to_cmudict.keys()
cmudict_keys = sorted(cmudict_keys, key=len, reverse=True)
espeak_regex = re.compile(rf"({'|'.join(cmudict_keys)})")

def cmudictify(espeak):
    espeak = espeak.replace("ˈ", "").replace("ˌ", "")
    return " ".join([espeak_to_cmudict[x] for x in re.findall(espeak_regex, espeak)])

In [46]:
text = "Yeah, that's true. I mean, they are the same size and they are a little bit, but I think I I should go more for something that style."
phones = "/jˈæ ðˈæs tɹˈuː ə mˈiːn ðˈeɪ ɚ ðə sˈeɪm sˈaɪz ən ðˈeɪ ɚ ə lˈɪɾə bˈɪɾ bˈʌt ˈaɪ θˈɪŋk ˈaɪ ˈaɪ ʃˈʊ ɡˈoʊ mˈɔːɹ fɚ sˈʌmθɪŋ ðˈæt stˈaɪl./"
EGFILE = "/Users/joregan/Playing/hsi/audio/hsi_5_0718_211_002_main.wav"
EGSTART = 70.028
EGEND = 75.441

In [ ]:
text = "no it's a cheetah, ah, yes it's a cheetah ah and, uh, that one is from, uh, India and, uh, it's really"
phones = "/nˈoʊ ɪts ɐ tʃˈiːt ˈɑːɛ jˈɑs ɪts ɐ tʃˈiːtə ˈɑː ænd ˈɛ ðˈæt wˌʌn ɪz fɹˈɑːm ˈɛ ˈɪndiæ ænd ˈɛ ɪts ɹˈili/"
EGFILE = "/Users/joregan/Playing/hsi/audio/hsi_7_0719_210_001_main.wav"

EGTEXT2 = "So it was standing somewhere in the middle of the room and it was flying around."
EGSTART2 = 122.709
EGEND2 = 126.344

In [7]:
def normword(text):
    text = text.strip(",.;:!?")
    return text.lower()

def normphon(phon):
    phon = phon.strip(",.;:!?")
    return phon

def make_lexicon(text, phon):
    if phon.startswith("/") and phon.endswith("/"):
        phon = phon[1:-1]
    words = [normword(x) for x in text.split(" ")]
    phonwords = [cmudictify(normphon(x)) for x in phon.split(" ")]
    assert len(words) == len(phonwords)
    output = list(set(zip(words, phonwords)))
    return output
    

In [12]:
def make_ps_dict(entries):
    counts = {}
    output = []
    lex = sorted(entries)
    for entry in lex:
        count = 1
        if not entry[0] in counts:
            counts[entry[0]] = 1
        else:
            counts[entry[0]] += 1
            count = counts[entry[0]]
        if count != 1:
            subscript = f"({count})"
        else:
            subscript = ""
        output.append(f"{entry[0]}{subscript} {entry[1]}")
    return output

In [13]:
def make_fsg_transitions_from_text(text):
    words = [normword(x) for x in text.split(" ")]
    enum = [x for x in enumerate(words)]
    trans = [(x[0], x[0] + 1, 1.0, x[1]) for x in enum]
    return trans

In [ ]:
audio = AudioSegment.from_file(EGFILE)
audio = audio.set_frame_rate(16000)

In [29]:
def align_with_pocketsphinx(text, phones, audio, start, end):
    lex = make_lexicon(text, phones)
    entries = make_ps_dict(lex)
    segments = []

    with (
        tempfile.NamedTemporaryFile(suffix=".dict") as dictf,
        tempfile.NamedTemporaryFile(suffix=".wav") as wavf,
    ):

        with open(dictf.name, "w") as dictout:
            for entry in entries:
                dictout.write(entry + "\n")

        audioseg = audio[int(start * 1000):int(end * 1000)]
        audioseg.export(wavf.name, format="wav", parameters=["-ac", "1", "-acodec", "pcm_s16le", "-ar", "16000"])


        fsgt = make_fsg_transitions_from_text(text)
        start_state = fsgt[0][0]
        end_state = fsgt[-1][1]

        decoder = pocketsphinx.Decoder(lm=None, dict=dictf.name)
        fsg = decoder.create_fsg("dummy", start_state, end_state, fsgt)
        decoder.add_fsg("dummy", fsg)
        decoder.activate_search("dummy")
        decoder.start_utt()
        # decoder.process_raw(seg.get_array_of_samples('B'))
        decoder.process_raw(wavf.read(), full_utt=True)
        decoder.end_utt()
        for seg in decoder.seg():
            segments.append({
                "word": seg.word,
                "start_frame": seg.start_frame,
                "end_frame": seg.end_frame
            })
    return segments

In [30]:
for seg in decoder.seg():
    print(seg)

In [21]:
with open("/tmp/mytmp.dict", "w") as dictout:
    for entry in entries:
        dictout.write(entry + "\n")


In [31]:
audioseg.export("/tmp/clip.wav", format="wav", parameters=["-ac", "1", "-acodec", "pcm_s16le", "-ar", "16000"])

<_io.BufferedRandom name='/tmp/clip.wav'>

In [32]:
psjson=!/opt/homebrew/bin/pocketsphinx align /tmp/clip.wav "yeah that's true i mean they are the same size and they are a little bit but i think i i should go more for something that style"

In [33]:
psjson

['{"b":0.000,"d":5.410,"p":1.000,"t":"yeah that\'s true i mean they are the same size and they are a little bit but i think i i should go more for something that style","w":[{"b":0.000,"d":0.250,"p":0.964,"t":"yeah"},{"b":0.250,"d":0.150,"p":0.937,"t":"that\'s"},{"b":0.400,"d":0.200,"p":0.978,"t":"true"},{"b":0.600,"d":0.060,"p":0.974,"t":"i"},{"b":0.660,"d":0.180,"p":0.980,"t":"mean"},{"b":0.840,"d":0.140,"p":0.979,"t":"they"},{"b":0.980,"d":0.070,"p":0.983,"t":"are(2)"},{"b":1.050,"d":0.110,"p":0.987,"t":"the"},{"b":1.160,"d":0.260,"p":0.974,"t":"same"},{"b":1.420,"d":0.380,"p":0.956,"t":"size"},{"b":1.800,"d":0.120,"p":0.976,"t":"and"},{"b":1.920,"d":0.110,"p":0.986,"t":"they"},{"b":2.030,"d":0.140,"p":0.983,"t":"are(2)"},{"b":2.170,"d":0.030,"p":0.990,"t":"a"},{"b":2.200,"d":0.180,"p":0.914,"t":"little"},{"b":2.380,"d":0.140,"p":0.980,"t":"bit"},{"b":2.520,"d":0.220,"p":0.934,"t":"but"},{"b":2.740,"d":0.110,"p":0.983,"t":"i"},{"b":2.850,"d":0.260,"p":0.968,"t":"think"},{"b":3.110,"

In [34]:
import json
data = json.loads(str(psjson[0]))

In [35]:
data

{'b': 0.0,
 'd': 5.41,
 'p': 1.0,
 't': "yeah that's true i mean they are the same size and they are a little bit but i think i i should go more for something that style",
 'w': [{'b': 0.0, 'd': 0.25, 'p': 0.964, 't': 'yeah'},
  {'b': 0.25, 'd': 0.15, 'p': 0.937, 't': "that's"},
  {'b': 0.4, 'd': 0.2, 'p': 0.978, 't': 'true'},
  {'b': 0.6, 'd': 0.06, 'p': 0.974, 't': 'i'},
  {'b': 0.66, 'd': 0.18, 'p': 0.98, 't': 'mean'},
  {'b': 0.84, 'd': 0.14, 'p': 0.979, 't': 'they'},
  {'b': 0.98, 'd': 0.07, 'p': 0.983, 't': 'are(2)'},
  {'b': 1.05, 'd': 0.11, 'p': 0.987, 't': 'the'},
  {'b': 1.16, 'd': 0.26, 'p': 0.974, 't': 'same'},
  {'b': 1.42, 'd': 0.38, 'p': 0.956, 't': 'size'},
  {'b': 1.8, 'd': 0.12, 'p': 0.976, 't': 'and'},
  {'b': 1.92, 'd': 0.11, 'p': 0.986, 't': 'they'},
  {'b': 2.03, 'd': 0.14, 'p': 0.983, 't': 'are(2)'},
  {'b': 2.17, 'd': 0.03, 'p': 0.99, 't': 'a'},
  {'b': 2.2, 'd': 0.18, 'p': 0.914, 't': 'little'},
  {'b': 2.38, 'd': 0.14, 'p': 0.98, 't': 'bit'},
  {'b': 2.52, 'd'

In [36]:
with open("/tmp/audacity.tsv", "w") as tsvf:
    for word in data["w"]:
        tsvf.write(f"{word['b']}\t{word['b']+word['d']}\t{word['t']}\n")

In [43]:
!/opt/homebrew/bin/sox /tmp/clip.wav $(/opt/homebrew/bin/pocketsphinx soxflags) > /tmp/ps.raw

In [44]:
from pocketsphinx import Pocketsphinx

psphone = Pocketsphinx(
    allphone="/opt/homebrew/Cellar/cmu-pocketsphinx/5.0.3/share/pocketsphinx/model/en-us/en-us-phone.lm.bin",
    lw=2.0,
    pip=0.3,
    beam=1e-200,
    pbeam=1e-20,
)
psphone.decode("/tmp/ps.raw")

In [45]:
psphone.hypothesis()

'SIL T Y AE D UH T UW AH M IH N EY Y ER D IH SH AO L SH AY SH ER M EY Y ER DH EH V IH D DH AH D EH TH IH NG G EH UW EY SH IH G UH OW W AO ER V F ER SH AH N D IH NG D EH CH AY AW L SIL'

In [ ]:
# jˈæ ðˈæs tɹˈuː ə mˈiːn ðˈeɪ ɚ ðə sˈeɪm sˈaɪz ən ðˈeɪ ˈɑːɹ ə lˈɪɾə bˈɪɾ bˈʌt ˈaɪ θˈɪŋk ˈaɪ ˈaɪ ʃˈʊ ɡˈoʊ mˈɔːɹ fɚ sˈʌmθɪŋ ðˈæt stˈaɪl.